In [1]:
import os
import pandas as pd

# 1. 원천 데이터 확인
모두의 말뭉치 신문 데이터 활용

신문 말뭉치 2023:
```
columns: Index(['file_id', 'doc_id', 'title', 'author', 'publisher', 'date', 'topic',
       'original_topic', 'sentence_id', 'sentence'],
      dtype='object')

Topic:
array(['정치', '사회', '생활', 'IT/과학', '경제', '미용/건강', '문화', '스포츠', '연예'],
      dtype=object)

샘플수:
File NEWSPAPER_2022_1.csv 1000000 Sentences 103464 Bodies
File NEWSPAPER_2022_2.csv 1000000 Sentences 110842 Bodies
File NEWSPAPER_2022_3.csv 1000000 Sentences 114289 Bodies
File NEWSPAPER_2022_4.csv 1000000 Sentences 123856 Bodies
File NEWSPAPER_2022_5.csv 1000000 Sentences 113822 Bodies
File NEWSPAPER_2022_6.csv 1000000 Sentences 110748 Bodies
File NEWSPAPER_2022_7.csv 1000000 Sentences 116875 Bodies
File NEWSPAPER_2022_8.csv 1000000 Sentences 110652 Bodies
File NEWSPAPER_2022_9.csv 992042 Sentences 118891 Bodies
```

In [2]:
# 모두의 말뭉치 - 신문 말뭉치 2023
dataset_name = "NIKL_NEWSPAPER_2023_CSV"
raw_source_dir = f"/Users/yrmbair/datasets/korean/modu/{dataset_name}"

fnames = [x for x in os.listdir(raw_source_dir) if "csv" in x]
fnames.sort()
fnames

['NEWSPAPER_2022_1.csv',
 'NEWSPAPER_2022_2.csv',
 'NEWSPAPER_2022_3.csv',
 'NEWSPAPER_2022_4.csv',
 'NEWSPAPER_2022_5.csv',
 'NEWSPAPER_2022_6.csv',
 'NEWSPAPER_2022_7.csv',
 'NEWSPAPER_2022_8.csv',
 'NEWSPAPER_2022_9.csv']

In [3]:
for fname in fnames:
    fpath = os.path.join(raw_source_dir, fname)
    df = pd.read_csv(fpath, encoding='utf-8')
    
    num_sentences = df.shape[0]
    num_bodies = len(df.doc_id.unique())
    print(f"File {fname} {num_sentences} Sentences {num_bodies} Bodies")

File NEWSPAPER_2022_1.csv 1000000 Sentences 103464 Bodies
File NEWSPAPER_2022_2.csv 1000000 Sentences 110842 Bodies
File NEWSPAPER_2022_3.csv 1000000 Sentences 114289 Bodies
File NEWSPAPER_2022_4.csv 1000000 Sentences 123856 Bodies
File NEWSPAPER_2022_5.csv 1000000 Sentences 113822 Bodies
File NEWSPAPER_2022_6.csv 1000000 Sentences 110748 Bodies
File NEWSPAPER_2022_7.csv 1000000 Sentences 116875 Bodies
File NEWSPAPER_2022_8.csv 1000000 Sentences 110652 Bodies
File NEWSPAPER_2022_9.csv 992042 Sentences 118891 Bodies


In [4]:
df.topic.unique()

array(['정치', '사회', '생활', 'IT/과학', '경제', '미용/건강', '문화', '스포츠', '연예'],
      dtype=object)

In [5]:
df.head()

,file_id,doc_id,title,author,publisher,date,topic,original_topic,sentence_id,sentence
0,NWRW2300000005,NWRW2300000005.4636,세계일보 2022년 기사,김현주,세계일보,20220201,정치,"정치>선거, 정치>국회_정당",NWRW2300000005.4636.12,한국리서치가 KBS 의뢰로 지난달 27~29일 이번 대선 인식을 조사한 결과 ‘정권...
1,NWRW2300000005,NWRW2300000005.4636,세계일보 2022년 기사,김현주,세계일보,20220201,정치,"정치>선거, 정치>국회_정당",NWRW2300000005.4636.13,정권교체를 희망하는 응답자들에게 야권 단일화 필요성을 물은 결과 59.4%가 필요하...
2,NWRW2300000005,NWRW2300000005.4636,세계일보 2022년 기사,김현주,세계일보,20220201,정치,"정치>선거, 정치>국회_정당",NWRW2300000005.4636.14,그러나 안 후보는 단일 후보로 나설 경우 이재명 후보를 16%p 이상 차이로 이기는...
3,NWRW2300000005,NWRW2300000005.4636,세계일보 2022년 기사,김현주,세계일보,20220201,정치,"정치>선거, 정치>국회_정당",NWRW2300000005.4636.15,윤 후보는 가급적 단일화 없이 승리를 가져오겠다는 구상이 확실하다. 당내에서는 이준...
4,NWRW2300000005,NWRW2300000005.4636,세계일보 2022년 기사,김현주,세계일보,20220201,정치,"정치>선거, 정치>국회_정당",NWRW2300000005.4636.16,선대기구를 새롭게 구성한 후 보이는 선거 전략은 공격적이면서 안정적이란 평가다. 이...


# 2. 데이터 전처리 코드 확인
- 문장 단위 -> 문단으로 재 취합
- sentence offset 기록

In [6]:

import json
import pandas as pd

# sentence_id에서 마지막 토큰(숫자) 기준으로 정렬하기 위한 helper
def sort_key(s):
    # 예: "NWRW2300000005.4636.12" -> 12
    return s.str.split('.').str[-1].astype(int)

def agg_file(group: pd.DataFrame) -> pd.Series:
    # sentence_id 순서대로 정렬
    group = group.sort_values('sentence_id', key=sort_key)

    sentences = group['sentence'].tolist()
    sentence_ids = group['sentence_id'].tolist()

    offsets = []
    text_parts = []
    pos = 0

    for i, sent in enumerate(sentences):
        # 문장 사이에 공백 하나씩 들어가므로 offset에 반영
        if i > 0:
            pos += 1  # 앞의 ' ' 한 칸
        start = pos
        text_parts.append(sent)
        pos += len(sent)
        end = pos
        offsets.append((start, end))

    text = " ".join(sentences)

    return pd.Series({
        "file_id": group['file_id'].iloc[0],
        "doc_id": group['doc_id'].iloc[0],
        "title": group['title'].iloc[0],
        "author": group['author'].iloc[0],
        "publisher": group['publisher'].iloc[0],
        "date": group['date'].iloc[0],
        "topic": group['topic'].iloc[0],
        "original_topic": group['original_topic'].iloc[0],
        # JSON string으로 저장 (한글 깨지지 않게 ensure_ascii=False)
        "sentence_ids": json.dumps(sentence_ids, ensure_ascii=False),
        "sentence_offsets": json.dumps(offsets),
        "text": text,
    })

def preprocess(df):
    agg_df = (
        df.groupby("doc_id", as_index=False)
        .apply(agg_file)
        .reset_index(drop=True)
    )
    agg_df = agg_df[
        [
            "file_id", "doc_id", "title", "author", "publisher",
            "date", "topic", "original_topic",
            "sentence_ids", "sentence_offsets", "text"
        ]
    ]
    return agg_df

In [7]:
agg_df = preprocess(df)

/var/folders/k4/hpklmk6s2vd2lgvfmg9wcm7h0000gn/T/ipykernel_30000/3134753007.py:50: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(agg_file)


In [8]:
print(agg_df.shape)
agg_df.head()

(118891, 11)


,file_id,doc_id,title,author,publisher,date,topic,original_topic,sentence_ids,sentence_offsets,text
0,NWRW2300000005,NWRW2300000005.10000,세계일보 2022년 기사,양다훈,세계일보,20220304,정치,"정치>선거, 정치>국회_정당, 정치>청와대","[""NWRW2300000005.10000.1"", ""NWRW2300000005.100...","[[0, 39], [40, 137], [138, 233], [234, 407], [...","이재명, 허경영 어록 인용 “나라에 돈이 없는 게 아니고 도둑이 많아” 이재명 더불..."
1,NWRW2300000005,NWRW2300000005.10001,세계일보 2022년 기사,장혜진,세계일보,20220304,정치,정치>선거,"[""NWRW2300000005.10001.1"", ""NWRW2300000005.100...","[[0, 35], [36, 280], [281, 462], [463, 673], [...",‘사전투표’ 가장 뜨거웠다… ‘양강’에 유불리 따지긴 아직 일러 제20대 대통령선거...
2,NWRW2300000005,NWRW2300000005.10002,세계일보 2022년 기사,김주영,세계일보,20220304,정치,"정치>청와대, 정치>선거","[""NWRW2300000005.10002.1"", ""NWRW2300000005.100...","[[0, 35], [36, 186], [187, 492], [493, 712], [...",투표 독려하며 ‘민주’ 세 번 언급한 文대통령… 野 “선거개입” 제20대 대통령선거...
3,NWRW2300000005,NWRW2300000005.10003,세계일보 2022년 기사,김현우,세계일보,20220304,정치,"정치>선거, 정치>국회_정당, 정치>청와대","[""NWRW2300000005.10003.1"", ""NWRW2300000005.100...","[[0, 24], [25, 130], [131, 408], [409, 657]]",심상정 “다당제로 바꿀 기회… 소신 투표를” 정의당 심상정 대선 후보는 사전 투표 ...
4,NWRW2300000005,NWRW2300000005.10004,세계일보 2022년 기사,김병관,세계일보,20220304,정치,"정치>선거, 정치>청와대, 정치>국회_정당","[""NWRW2300000005.10004.1"", ""NWRW2300000005.100...","[[0, 35], [36, 115], [116, 200], [201, 366], [...",野 최승재 “이재명 대통령 되면 경험하지 못한 나라 계속될 것” 국민의힘 최승재 의...


In [9]:
agg_df.iloc[0]

file_id                                                NWRW2300000005
doc_id                                           NWRW2300000005.10000
title                                                   세계일보 2022년 기사
author                                                            양다훈
publisher                                                        세계일보
date                                                         20220304
topic                                                              정치
original_topic                                정치>선거, 정치>국회_정당, 정치>청와대
sentence_ids        ["NWRW2300000005.10000.1", "NWRW2300000005.100...
sentence_offsets    [[0, 39], [40, 137], [138, 233], [234, 407], [...
text                이재명, 허경영 어록 인용 “나라에 돈이 없는 게 아니고 도둑이 많아” 이재명 더불...
Name: 0, dtype: object

In [10]:
text = agg_df.iloc[0]['text']
print(len(text))
print(text)

967
이재명, 허경영 어록 인용 “나라에 돈이 없는 게 아니고 도둑이 많아” 이재명 더불어민주당 대선후보가 허경영 국가혁명당 대선후보의 어록을 빌려 “누가 그러지 않았나. 저도 동의하는 말인데 나라에 돈이 없는 게 아니고 도둑이 너무 많다”고 말했다. 이날 오후 강원도 유세에서 이 후보는 “똑같은 성남시 예산을 가지고 빚지거나 세금을 올리지 않고 7500억원의 부채를 제가 3년 6개월 만에 정리했다”며 이같이 지적했다. 이 후보는 국민의힘 측이 대장동 사건 관련 공세를 펴는 것에 “선량한 도둑을 잡는 사람한테 도둑이라고 뒤집어씌우더라. 이게 정치다”라며 “정치가 있는 힘을 다 모아서 국민이 맡긴 일을 제대로 하는 게 본령인데 자기 사욕이나 채우고, 제 주머니 채우다가 그것을 막는 선량한 정치가 있으면 퇴출시킨다”고 비판했다. 그러면서 “자신을 위해 정치하는 잘못된 정치가 우리의 삶을 이 정도 밖에 못 만든 것”이라며 “정치만 똑바로 하면 나라를 완전히 새로 만들 수 있다”고 강조했다. 이 후보는 “정치인들이 발목 잡아서, 상대를 실수하게 해서 나의 기회를 찾는 적대적 공생의 정치 말고 선의의 경쟁이 가능한, 발목잡기 안 할 수 있는 정치를 만들면 나쁜 짓 안 하는 정치인이 성공할 것이고, 정치 역량이 국민 삶을 개선하는 데 쓰일 것 아니냐”고 반문했다. 이어 “그것이 제가 대통령이 되는 것보다 중요하다. 정치개혁을 통해 다당제를 하고, 거대 양당 두 개가 아니라 제3, 제4의 선택이 가능한 정치구조를 만들려는 것인 이번 선거에서 도움받으려는 전략이 아니고 저 이재명이 평생 가진 꿈이었다”고 강조했다. 또한 “(실제로) 개인 돈은 아니고 국가의 돈인데 이 돈을 어디에 쓸 것인지 결정할 권한을 가질 사람을 뽑는다”며 “다른 생각하는 (후보를) 잘못 뽑으면 지가 마음대로 돈을 쓸 수 있다. 방위 비리를 저지르고 4대강 하는데 22조원을 쓰고, 이런 데 막 쓰면 우리를 위해 쓸 돈이 없지 않냐”며 지지를 호소했다.


# 3. 전처리

```
NEWSPAPER_2022_1.csv aggregated to 103464 Texts
NEWSPAPER_2022_2.csv aggregated to 110842 Texts
NEWSPAPER_2022_3.csv aggregated to 114289 Texts
NEWSPAPER_2022_4.csv aggregated to 123856 Texts
NEWSPAPER_2022_5.csv aggregated to 113822 Texts
NEWSPAPER_2022_6.csv aggregated to 110748 Texts
NEWSPAPER_2022_7.csv aggregated to 116875 Texts
NEWSPAPER_2022_8.csv aggregated to 110652 Texts
NEWSPAPER_2022_9.csv aggregated to 118891 Texts
```

In [11]:
if not os.path.exists("source"):
    os.makedirs("source")

if not os.path.exists(f"source/{dataset_name}"):
    os.makedirs(f"source/{dataset_name}")

In [13]:
for fname in fnames:
    fpath = os.path.join(raw_source_dir, fname)
    df = pd.read_csv(fpath, encoding='utf-8')
    
    agg_df = preprocess(df)
    print(f"{fname} aggregated to {agg_df.shape[0]} Texts")
    
    ## Save 
    file_id = fname.replace(".csv", "")
    agg_df.to_parquet(f"source/{dataset_name}/{file_id}.parquet")

/var/folders/k4/hpklmk6s2vd2lgvfmg9wcm7h0000gn/T/ipykernel_30000/3134753007.py:50: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(agg_file)


NEWSPAPER_2022_1.csv aggregated to 103464 Texts


/var/folders/k4/hpklmk6s2vd2lgvfmg9wcm7h0000gn/T/ipykernel_30000/3134753007.py:50: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(agg_file)


NEWSPAPER_2022_2.csv aggregated to 110842 Texts


/var/folders/k4/hpklmk6s2vd2lgvfmg9wcm7h0000gn/T/ipykernel_30000/3134753007.py:50: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(agg_file)


...

NEWSPAPER_2022_7.csv aggregated to 116875 Texts


/var/folders/k4/hpklmk6s2vd2lgvfmg9wcm7h0000gn/T/ipykernel_30000/3134753007.py:50: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(agg_file)


NEWSPAPER_2022_8.csv aggregated to 110652 Texts


/var/folders/k4/hpklmk6s2vd2lgvfmg9wcm7h0000gn/T/ipykernel_30000/3134753007.py:50: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(agg_file)


NEWSPAPER_2022_9.csv aggregated to 118891 Texts
